<a href="https://colab.research.google.com/github/jatinkumar4148/Travel_Guide_Assistant/blob/main/LANG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q uninstall -y transformers tokenizers accelerate sentence-transformers


In [2]:
!pip -q install transformers==4.38.2 tokenizers==0.15.2 accelerate==0.27.2 sentence-transformers==2.6.1
!pip -q install streamlit faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.6 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

travel_guides = {
    "London": (
        "London travel guide: British Museum, Tower of London, Buckingham Palace, London Eye, Westminster Abbey, River Thames cruise. "
        "Food: fish and chips, Sunday roast, Indian curry, afternoon tea. "
        "Transport: The Tube (Underground) is efficient; use an [Oyster card](https://tfl.gov.uk). Tips: stand on the right of escalators, book popular attractions in advance."
    ),
    "Rome": (
        "Rome travel guide: Colosseum, Pantheon, Trevi Fountain, Roman Forum, Vatican City, St. Peter's Basilica. "
        "Food: Carbonara, Cacio e Pepe, Roman-style pizza, gelato. "
        "Transport: Metro and buses are best. Tips: dress modestly in churches, book [Vatican Museum tickets](https://m.museivaticani.va) weeks ahead."
    ),
    "Barcelona": (
        "Barcelona travel guide: Sagrada Família, Park Güell, Casa Batlló, Gothic Quarter, La Rambla, Barceloneta Beach. "
        "Food: Paella, tapas, Bombas, Pa amb Tomàquet. "
        "Transport: Extensive Metro and bus network. Tips: beware of pickpockets in crowded spots, book Gaudí sites online to skip lines."
    ),
    "Bangkok": (
        "Bangkok travel guide: Grand Palace, Wat Pho (Reclining Buddha), Wat Arun, Chatuchak Weekend Market, Khao San Road. "
        "Food: Pad Thai, Tom Yum Goong, mango sticky rice, street food. "
        "Transport: BTS Skytrain and MRT are best. Tips: dress respectfully for temples, use the [Grab app](https://www.grab.com) for rides."
    ),
    "Dubai": (
        "Dubai travel guide: Burj Khalifa, Dubai Mall, Palm Jumeirah, Desert Safari, Old Dubai Gold Souk, Museum of the Future. "
        "Food: Shawarma, Manousheh, Al Harees, Luqaimat. "
        "Transport: Dubai Metro and taxis; get a [Nol Card](https://www.rta.ae). Tips: mall walking involves long distances, stay hydrated."
    ),
    "Singapore": (
        "Singapore travel guide: Gardens by the Bay, Marina Bay Sands, Sentosa Island, Universal Studios, Merlion Park. "
        "Food: Hainanese chicken rice, Laksa, Chilli crab, Satay. "
        "Transport: Highly efficient MRT; use the [SimplyGo app](https://simplygo.com.sg). Tips: chewing gum is restricted, hawker centers offer the best food deals."
    ),
    "Istanbul": (
        "Istanbul travel guide: Hagia Sophia, Blue Mosque, Topkapi Palace, Grand Bazaar, Bosphorus Cruise, Galata Tower. "
        "Food: Baklava, Doner kebab, Turkish delight, Mezes. "
        "Transport: Trams and ferries are great; use an [Istanbulkart](https://www.istanbulkart.istanbul). Tips: learn basic greetings, haggle in the bazaars."
    ),
    "Seoul": (
        "Seoul travel guide: Gyeongbokgung Palace, Myeongdong shopping district, N Seoul Tower, Bukchon Hanok Village, Gangnam. "
        "Food: Bibimbap, Korean BBQ, Kimchi, tteokbokki. "
        "Transport: Excellent Subway system; use a [T-money card](https://english.visitkorea.or.kr). Tips: use [Naver Maps](https://map.naver.com) for navigation."
    ),
    "Amsterdam": (
        "Amsterdam travel guide: Anne Frank House, Rijksmuseum, Van Gogh Museum, Canal Cruise, Dam Square, Jordaan district. "
        "Food: Stroopwafels, Dutch fries, bitterballen, raw herring. "
        "Transport: Cycling is the primary mode; use an [OV-chipkaart](https://www.ov-chipkaart.nl). Tips: stay out of bike lanes, book museums months in advance."
    ),
    "Switzerland": (
        "Switzerland travel guide: Visit the Matterhorn in Zermatt, Lake Lucerne, Interlaken (Jungfrau region), Rhine Falls, and Bern. "
        "Food: cheese fondue, raclette, Rösti, Swiss chocolate. "
        "Transport: Use the world-class [SBB rail network](https://www.sbb.ch). Tips: stay near stations, download the SBB app for real-time schedules."
    ),
    "Berlin": (
        "Berlin travel guide: Brandenburg Gate, Reichstag Building, Berlin Wall Memorial, Museum Island, East Side Gallery. "
        "Food: Currywurst, Döner kebab, Schnitzel, Berliner Pfannkuchen. "
        "Transport: Efficient U-Bahn and S-Bahn network. Tips: carry cash for small shops, most stores are closed on Sundays."
    ),
    "Los Angeles": (
        "Los Angeles travel guide: Griffith Observatory, Hollywood Walk of Fame, Santa Monica Pier, Getty Center, Universal Studios. "
        "Food: street tacos, In-N-Out Burger, Korean BBQ, fresh seafood. "
        "Transport: A car is highly recommended; Metro Rail is limited. Tips: book free tickets for [The Getty](https://www.getty.edu) early."
    ),
    "Shanghai": (
        "Shanghai travel guide: The Bund skyline, Yu Garden, Shanghai Tower, Nanjing Road shopping, French Concession. "
        "Food: Xiaolongbao (soup dumplings), Shengjian mantu, hairy crab. "
        "Transport: Extensive Metro system; use [Alipay](https://www.alipay.com) for all payments. Tips: carry your passport for sightseeing."
    ),
    "Moscow": (
        "Moscow travel guide: Red Square, Saint Basil's Cathedral, The Kremlin, Gorky Park, Bolshoi Theatre. "
        "Food: Borscht, Pelmeni, Blini, Beef Stroganoff. "
        "Transport: The Moscow Metro is fast and artistic; use a Troika card. Tips: learn basic Cyrillic to read signs, dress warmly in winter."
    ),
    "Luxembourg": (
        "Luxembourg City travel guide: Bock Casemates, Grand Ducal Palace, Chemin de la Corniche, Notre-Dame Cathedral. "
        "Food: Judd mat Gaardebounen, Gromperekichelcher, Quetschentaart. "
        "Transport: All public transport is [free nationwide](https://www.luxembourg-city.com). Tips: wear comfortable shoes for hills."
    ),
    "Tokyo": (
        "Tokyo travel guide: Shibuya Crossing, Senso-ji Temple, Meiji Shrine, Tokyo Skytree, Akihabara electronics district. "
        "Food: Ramen, Sushi, Tempura, Wagyu beef. "
        "Transport: World-class Metro and JR lines; use a Suica card. Tips: follow train etiquette, carry a bag for your trash."
    ),
    "Sydney": (
        "Sydney travel guide: Sydney Opera House, Harbour Bridge, Bondi Beach, Darling Harbour, The Rocks. "
        "Food: Meat pies, Barramundi, Pavlova, Lamingtons. "
        "Transport: Trains and ferries; use an [Opal card](https://transportnsw.info). Tips: wear strong sunscreen (SPF 50+)."
    ),
    "Maharashtra": (
            "Maharashtra travel guide: Gateway of India, Marine Drive, Elephanta Caves, Ajanta and Ellora Caves, Mahabaleshwar hill station. "
            "Food: Vada Pav, Pav Bhaji, Misal Pav, Puran Poli. "
            "Transport: Local trains in Mumbai, state buses (MSRTC), and the new Atal Setu bridge. Tips: avoid local trains during peak hours, visit Konkan coast during winters."
   ),
    "Delhi": (
            "Delhi travel guide: Red Fort, Qutub Minar, India Gate, Lotus Temple, Humayun's Tomb, Akshardham Temple. "
            "Food: Butter chicken, Chole Bhature, Aloo Chaat, Paranthas. "
            "Transport: Delhi Metro is world-class; use the DMRC app. Tips: use the One Delhi app for bus tracking, shop at Sarojini Nagar for bargains."
        ),
        "Kashmir": (
            "Kashmir travel guide: Dal Lake Shikara ride, Gulmarg Gondola, Pahalgam (Betaab Valley), Sonamarg, Shalimar Bagh. "
            "Food: Wazwan, Rogan Josh, Kahwa tea, Yakhni. "
            "Transport: Private taxis are best for mountain travel. Tips: carry heavy woollens even in spring, check for current travel permits for border areas."
        ),
        "Meghalaya": (
            "Meghalaya travel guide: Living Root Bridges in Cherrapunji, Dawki River (Umngot), Mawlynnong (cleanest village), Elephant Falls. "
            "Food: Jadoh, Doh-Khlieh, Nakham Bitchi. "
            "Transport: Taxis from Guwahati or Shillong are the main mode. Tips: always carry a raincoat, respect the sacred groves of the Khasi hills."
        ),
        "Assam": (
            "Assam travel guide: Kaziranga National Park (One-horned Rhino), Kamakhya Temple, Majuli River Island, Tea Gardens of Dibrugarh. "
            "Food: Masor Tenga (sour fish), Khar, Duck meat curry, Pitha. "
            "Transport: Well-connected by rail; use ferries to reach Majuli. Tips: visit in April for the Bihu festival, book safaris at least a month early."
        ),
        "Rajasthan": (
            "Rajasthan travel guide: Jaipur's Hawa Mahal, Udaipur's Lake Palace, Jaisalmer Sand Dunes, Jodhpur's Mehrangarh Fort. "
            "Food: Dal Baati Churma, Laal Maas, Ker Sangri, Ghevar. "
            "Transport: Trains connect all major cities; e-rickshaws are best for narrow streets. Tips: stay in a 'Haveli' for a royal experience, carry sunscreen."
        ),
        "Goa": (
            "Goa travel guide: Baga and Calangute beaches, Old Goa Churches, Dudhsagar Falls, South Goa peaceful retreats. "
            "Food: Fish Recheado, Bebinca, Pork Vindaloo, Feni. "
            "Transport: Renting a scooty or car is the most popular way. Tips: use the Goa Miles app for official taxi rates, avoid swimming in the sea during monsoons."
        ),
        "Kerala": (
            "Kerala travel guide: Munnar Tea Estates, Alleppey Backwaters (Houseboats), Wayanad forests, Fort Kochi. "
            "Food: Appam with Stew, Kerala Sadya on banana leaf, Karimeen Pollichathu. "
            "Transport: KSRTC buses and the new Kochi Water Metro. Tips: book houseboats via government-approved vendors, carry mosquito repellent."
        ),
        "Uttar Pradesh": (
            "Uttar Pradesh travel guide: Taj Mahal in Agra, Ganga Aarti in Varanasi, Lucknow's Bara Imambara, Sangam in Prayagraj. "
            "Food: Galouti Kebab, Tunday Kababi, Banarasi Paan, Petha. "
            "Transport: Fast trains like Vande Bharat; rickshaws in old cities. Tips: arrive at the Taj Mahal at sunrise, be wary of unofficial guides in Varanasi."
        ),
        "West Bengal": (
            "West Bengal travel guide: Victoria Memorial in Kolkata, Darjeeling Toy Train, Sundarbans Mangrove Forest, Howrah Bridge. "
            "Food: Machher Jhol (fish curry), Rosogolla, Mishti Doi, Jhalmuri. "
            "Transport: Kolkata Metro and iconic yellow taxis. Tips: experience Durga Puja in October for local culture, book Darjeeling hotels early."
        )
    }

@st.cache_resource
def load_embedder():
    return SentenceTransformer("all-MiniLM-L6-v2")

@st.cache_resource
def load_llm():
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

def llm_generate(prompt: str) -> str:
    tokenizer, model = load_llm()
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_new_tokens=250, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

@st.cache_resource
def build_faiss_index():
    embedder = load_embedder()
    cities = list(travel_guides.keys())
    docs = [travel_guides[c] for c in cities]
    vectors = embedder.encode(docs, convert_to_numpy=True).astype("float32")

    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    return index, cities, docs

def retrieve_docs(query, k=1):
    embedder = load_embedder()
    index, cities, docs = build_faiss_index()
    q_vec = embedder.encode([query], convert_to_numpy=True).astype("float32")
    D, I = index.search(q_vec, k)
    return [(cities[i], docs[i]) for i in I[0]]

def generate_itinerary(city, days, budget, preferences):
    query = f"{city} {days} day itinerary {preferences} budget {budget}"
    retrieved = retrieve_docs(query, k=1)
    context = "\n".join([f"{c}: {t}" for c, t in retrieved])

    prompt = f"""
Use only this context:
{context}

Make a {days}-day itinerary for {city}.
Preferences: {preferences}
Budget: {budget}
Give bullet points day-wise.
Add 3 local tips.
"""
    return llm_generate(prompt), retrieved

st.title("🌍 Travel Guide Assistant (RAG)")

city = st.selectbox("Destination", list(travel_guides.keys()))
days = st.number_input("Days", 1, 10, 3)
budget = st.number_input("Budget ($)", 100, 10000, 1500)
preferences = st.text_input("Preferences", "food and culture")

if st.button("Generate Plan"):
    ans, src = generate_itinerary(city, days, budget, preferences)
    st.subheader("Itinerary")
    st.write(ans)

    st.subheader("Sources")
    for c, t in src:
        st.write(c, ":", t)


Writing app.py


In [4]:
!pkill -f streamlit
!streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false --server.headless true &>/content/logs.txt &


In [5]:
from google.colab import output
output.serve_kernel_port_as_iframe(8501)


<IPython.core.display.Javascript object>